In [1]:
print("hello world")

hello world


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor, BaggingClassifier, BaggingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout

2023-11-03 17:02:50.289213: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-03 17:02:50.389521: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Loading dataset

In [40]:
Emotion = pd.read_csv('./data_sources/features.csv')
Emotion.head() 

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,labels
0,-637.701233,104.299019,4.894947,20.494011,12.552954,2.851410,-6.633390,-4.091278,-10.423918,-6.406950,...,0.172893,-1.170210,-5.292450,-0.573319,1.019471,-3.492607,-3.468123,-1.214944,1.971239,male_calm
1,-596.908460,86.871936,9.470162,17.109819,11.198966,1.541056,-6.677264,-5.755428,-9.684472,-6.891256,...,0.033803,-1.986515,-5.103855,-1.253110,0.514896,-3.268317,-4.502895,0.167153,0.256732,male_calm
2,-698.086548,99.795929,1.892679,19.915264,7.532868,1.265761,-9.188656,-5.798194,-12.299710,-4.976400,...,-1.639542,-2.603761,-4.890347,-0.879222,-1.250208,-3.449960,-4.708529,-0.086224,-3.034044,male_calm
3,-279.141052,41.092949,-21.319229,7.802911,-13.140503,-9.407660,-15.580647,-6.097223,-24.700903,-9.640293,...,-10.305976,-4.092835,-5.817179,-10.731523,-0.823596,-15.885103,-2.014258,-6.173852,-5.331760,male_angry
4,-160.074686,17.576058,-2.147436,3.133417,-4.745002,-6.510771,-5.911591,-4.481506,-9.470598,-5.907823,...,-4.088007,-1.817639,-2.994765,-4.893176,-2.880436,-7.163400,-2.147345,-2.359248,-3.746217,male_angry


### Extract features and labels

In [41]:
X = Emotion.iloc[:, :-1].values
y = Emotion['labels'].values

In [42]:
y


array(['male_calm', 'male_calm', 'male_calm', ..., 'female_disgust',
       'female_disgust', 'female_disgust'], dtype=object)

In [43]:
import numpy as np
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(np.array(y).reshape(-1,1)).toarray()

### Split the data into train and test sets

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0, shuffle=True)


### Scaling the features

In [45]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Defining the random search hyperparameter grids for classifiers and regressors

### Classifiers

In [9]:
classifiers = {
    'SVC': {
        'model': SVC(),
        'param_dist': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto', 0.001, 0.01]}
    },
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'param_dist': {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    },
    'GradientBoostingClassifier': {
        'model': GradientBoostingClassifier(),
        'param_dist': {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 0.2]}
    },
    'KNeighborsClassifier': {
        'model': KNeighborsClassifier(),
        'param_dist': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
    },
    'MLPClassifier': {
        'model': MLPClassifier(),
        'param_dist': {'hidden_layer_sizes': [(50, 25), (100, 50, 25), (50, 50, 25)], 'activation': ['relu', 'logistic'], 'alpha': [0.0001, 0.001, 0.01]}
    },
    'BaggingClassifier': {
        'model': BaggingClassifier(),
        'param_dist': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 0.7, 0.9]}
    }
}


### Regressors

In [10]:
regressors = {
    'SVR': {
        'model': SVR(),
        'param_dist': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 
                       'gamma': ['scale', 'auto', 0.001, 0.01]}
    },
    'RandomForestRegressor': {
        'model': RandomForestRegressor(),
        'param_dist': {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30], 
                       'min_samples_split': [2, 5, 10]}
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(),
        'param_dist': {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 
                       'learning_rate': [0.01, 0.1, 0.2]}
    },
    'KNeighborsRegressor': {
        'model': KNeighborsRegressor(),
        'param_dist': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
    },
    'MLPRegressor': {
        'model': MLPRegressor(),
        'param_dist': {'hidden_layer_sizes': [(50, 25), (100, 50, 25), (50, 50, 25)], 
                       'activation': ['relu', 'logistic'], 'alpha': [0.0001, 0.001, 0.01]}
    },
    'BaggingRegressor': {
        'model': BaggingRegressor(),
        'param_dist': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 0.7, 0.9]}
    }
}

### Randomized Search for classifiers

In [11]:
for name, clf in classifiers.items():
    print(f'Classifier: {name}')
    rand_search = RandomizedSearchCV(estimator=clf['model'], param_distributions=clf['param_dist'], 
                                     n_iter=10, scoring='accuracy', cv=3, random_state=0)
    rand_search.fit(X_train, y_train)
    
    # Print the best parameters and accuracy
    print(f'Best Parameters: {rand_search.best_params_}')
    print(f'Best Accuracy: {rand_search.best_score_:.2f}\n')

Classifier: SVC
Best Parameters: {'kernel': 'rbf', 'gamma': 0.01, 'C': 10}
Best Accuracy: 0.62

Classifier: RandomForestClassifier
Best Parameters: {'n_estimators': 300, 'min_samples_split': 2, 'max_depth': None}
Best Accuracy: 0.65

Classifier: GradientBoostingClassifier
Best Parameters: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.2}
Best Accuracy: 0.61

Classifier: KNeighborsClassifier


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Parameters: {'weights': 'distance', 'n_neighbors': 3}
Best Accuracy: 0.68

Classifier: MLPClassifier


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best Parameters: {'hidden_layer_sizes': (100, 50, 25), 'alpha': 0.001, 'activation': 'relu'}
Best Accuracy: 0.68

Classifier: BaggingClassifier
Best Parameters: {'n_estimators': 100, 'max_samples': 0.9}
Best Accuracy: 0.59


### Random Search for Regressors

In [12]:
for name, reg in regressors.items():
    print(f'Regressor: {name}')
    rand_search = RandomizedSearchCV(estimator=reg['model'], param_distributions=reg['param_dist'], n_iter=10, scoring='neg_mean_squared_error', cv=3, random_state=0)
    rand_search.fit(X_train, y_train)
    
    # Calculate RMSE
    y_pred = rand_search.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    # Print the best parameters and RMSE
    print(f'Best Parameters: {rand_search.best_params_}')
    print(f'RMSE: {rmse:.2f}\n')

Regressor: SVR
Best Parameters: {'kernel': 'rbf', 'gamma': 'auto', 'C': 1}
RMSE: 2.49

Regressor: RandomForestRegressor
Best Parameters: {'n_estimators': 300, 'min_samples_split': 2, 'max_depth': None}
RMSE: 2.39

Regressor: GradientBoostingRegressor
Best Parameters: {'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.1}
RMSE: 2.35

Regressor: KNeighborsRegressor


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Parameters: {'weights': 'distance', 'n_neighbors': 3}
RMSE: 1.63

Regressor: MLPRegressor


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best Parameters: {'hidden_layer_sizes': (50, 25), 'alpha': 0.01, 'activation': 'relu'}
RMSE: 1.97

Regressor: BaggingRegressor
Best Parameters: {'n_estimators': 100, 'max_samples': 0.9}
RMSE: 2.43


In [240]:
mlp = MLPClassifier(hidden_layer_sizes=(400, 200, 150), max_iter=500, random_state=42, alpha=0.001, batch_size=32,
                    learning_rate='constant', activation='relu', solver='adam', shuffle=True)
mlp.fit(X_train, y_train)

MLPClassifier(alpha=0.001, batch_size=32, hidden_layer_sizes=(400, 200, 150),
              max_iter=500, random_state=42)

In [241]:
print("Training set score: {:.3f}".format(mlp.score(X_train, y_train)))
print("Test set score: {:.3f}".format(mlp.score(X_test, y_test)))

Training set score: 1.000
Test set score: 0.767


In [174]:
import IPython.display as ipd
from IPython.display import Audio
RAVD = "data_sources/RAVDESS/audio_speech_actors_01-24/"
audio_path = RAVD + 'Actor_21/03-01-08-02-01-01-21.wav' # Gender - male; Emotion - suprised
ipd.Audio(audio_path)

In [175]:
import librosa
#import numpy as np
data, sample_rate = librosa.load(audio_path, duration=2.5, offset=0.5)
mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
# normal data
res1 = mfccs
result = np.array(res1)

In [176]:
result

array([-426.42996   ,   68.09493   ,  -11.61861   ,   15.92971   ,
          1.7999741 ,   -9.967348  ,  -11.8653345 ,   -5.8879147 ,
        -25.89691   ,    0.53543544,  -12.041248  ,  -13.766     ,
         -4.7922983 ,   -3.677003  ,   -7.4496765 ,   -2.871484  ,
        -11.560259  ,   -2.3647187 ,   -3.7733357 ,   -4.397298  ],
      dtype=float32)

In [177]:
result = result.reshape(1, -1)

In [178]:
result

array([[-426.42996   ,   68.09493   ,  -11.61861   ,   15.92971   ,
           1.7999741 ,   -9.967348  ,  -11.8653345 ,   -5.8879147 ,
         -25.89691   ,    0.53543544,  -12.041248  ,  -13.766     ,
          -4.7922983 ,   -3.677003  ,   -7.4496765 ,   -2.871484  ,
         -11.560259  ,   -2.3647187 ,   -3.7733357 ,   -4.397298  ]],
      dtype=float32)

In [179]:
scaled_features = scaler.transform(result)

In [180]:
predicted_emotion = mlp.predict(scaled_features)

In [181]:
predicted_emotion

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [182]:
predicted_emotion = (encoder.inverse_transform(predicted_emotion))

In [183]:
print(f'Predicted Emotion: {predicted_emotion[0]}')

Predicted Emotion: ['male_surprise']
